In [148]:
#Import Libraries

from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [149]:
#Configuration Jobs

#spark = SparkSession \
#    .builder \
#    .appName("Flights") \
#    .getOrCreate()

spark.conf.set('spark.sql.repl.eagerEval.enabled',True)

In [150]:
#Load csv Data to DataFrame

In [151]:
#Flights
flightsDF= spark.read.option("header","true").csv("./Data/flights.csv")
#Airports
airportsDF= spark.read.option("header","true").csv("./Data/airports.csv")
#Airline
airlinesDF= spark.read.option("header","true").csv("./Data/airlines.csv")

In [152]:
#Functions to transform or remove columns
def RemoveUnnecessaryColumnsFunction(df: DataFrame) -> DataFrame:
    columns_to_drop = [
                        'ORIGIN_AIRPORT',
                        'IATA_CODE',
                        'AIRPORT',
                        'CITY',
                        'STATE',
                        'COUNTRY',
                        'LATITUDE',
                        'LONGITUDE']
    return df.drop(*columns_to_drop)

In [153]:
#join airlines to flights to avoid code
flightsDF = flightsDF.join(airlinesDF,flightsDF.AIRLINE ==  airlinesDF.IATA_CODE,"inner").drop(flightsDF.AIRLINE).drop(col("IATA_CODE")).withColumnRenamed("AIRLINE","AIRLINE_") 
flightsDF = flightsDF.join(airportsDF,flightsDF.ORIGIN_AIRPORT ==  airportsDF.IATA_CODE,"inner").withColumnRenamed("CITY","CITY_DEPART_") 
flightsDF = RemoveUnnecessaryColumnsFunction(flightsDF)
flightsDF = flightsDF.join(airportsDF,flightsDF.DESTINATION_AIRPORT ==  airportsDF.IATA_CODE,"inner").withColumnRenamed("CITY","CITY_ARRIVAL_").drop(col("DESTINATION_AIRPORT")) 
flightsDF = RemoveUnnecessaryColumnsFunction(flightsDF)

In [154]:
#flightsDF.select('FLIGHT_NUMBER').distinct().count()  # 6327 num de vol unique
#flightsDF.groupBy('FLIGHT_NUMBER').count().orderBy('count', ascending=False)

FLIGHT_NUMBER,count
345,230
469,230
711,213
341,209
404,208
61,207
745,201
12,199
188,198
432,197


In [155]:
#Transform Date

flightsDF = flightsDF.withColumn('DATE_', to_date(concat_ws('-',flightsDF.YEAR,flightsDF.MONTH,flightsDF.DAY))).drop(*['YEAR','MONTH','DAY','DAY_OF_WEEK'])
flightsDF = RemoveUnnecessaryColumnsFunction(flightsDF);
flightsDF.sample(0.1)


FLIGHT_NUMBER,TAIL_NUMBER,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,AIRLINE_,CITY_DEPART_,CITY_ARRIVAL_,DATE_
840,N171US,0020,0018,-2,16,0034,286,293,266,2296,0800,11,0806,0811,5,0,0,null,null,null,null,null,null,US Airways Inc.,San Francisco,Charlotte,2015-01-01
258,N3HYAA,0020,0015,-5,15,0030,285,281,258,2342,0748,8,0805,0756,-9,0,0,null,null,null,null,null,null,American Airlines...,Los Angeles,Miami,2015-01-01
2440,N651DL,0040,0039,-1,28,0107,189,198,166,1399,0553,4,0549,0557,8,0,0,null,null,null,null,null,null,Delta Air Lines Inc.,Seattle,Minneapolis,2015-01-01
1670,N806DN,0050,0045,-5,9,0054,193,186,171,1426,0545,6,0603,0551,-12,0,0,null,null,null,null,null,null,Delta Air Lines Inc.,Portland,Minneapolis,2015-01-01
972,N606NK,0159,0158,-1,11,0209,123,125,103,868,0452,11,0502,0503,1,0,0,null,null,null,null,null,null,Spirit Air Lines,Phoenix,Dallas-Fort Worth,2015-01-01
2134,N307JB,0400,0535,95,9,0544,185,175,163,1189,0727,3,0605,0730,85,0,0,null,0,0,85,0,0,JetBlue Airways,San Juan,Orlando,2015-01-01
730,N423AS,0505,0457,-8,16,0513,205,199,179,1448,0912,4,0930,0916,-14,0,0,null,null,null,null,null,null,Alaska Airlines Inc.,Anchorage,Seattle,2015-01-01
7419,N445SW,0510,0508,-2,21,0529,69,67,41,257,0610,5,0619,0615,-4,0,0,null,null,null,null,null,null,Skywest Airlines ...,Aberdeen,Minneapolis,2015-01-01
1279,N918FR,0530,0526,-4,12,0538,160,146,128,861,0746,6,0810,0752,-18,0,0,null,null,null,null,null,null,Frontier Airlines...,Philadelphia,Orlando,2015-01-01
5184,N776SK,0540,0537,-3,18,0555,193,180,158,1195,0933,4,0953,0937,-16,0,0,null,null,null,null,null,null,Skywest Airlines ...,Salt Lake City,Houston,2015-01-01
